In [1]:
import librosa
import numpy as np
import soundfile as sf
import os
import pandas as pd

def augment_audio(audio_path, output_path, augment_type='speed'):
    y, sr = librosa.load(audio_path)
    #tipo de aumentacion
    # Aumentación: Cambio de velocidad
    if augment_type == 'speed':
        y_aug = librosa.effects.time_stretch(y, rate=np.random.uniform(0.9, 1.1))
    
    # Aumentación: Cambio de tono
    elif augment_type == 'pitch':
        y_aug = librosa.effects.pitch_shift(y, sr=sr, n_steps=np.random.randint(-2, 3))
    # Aumentación: Cambio de tono
    elif augment_type == 'distortion':
        y_aug = y + 0.3 * np.sin(np.linspace(0, np.pi * 2, len(y)))
    #se pueden agregar mas aumentaicones
    
    # Guardar el audio aumentado con soundfile
    sf.write(output_path, y_aug, sr)

def augment_class(csv_path, class_to_augment, augment_counts, new_folders, new_csv_path):

    df = pd.read_csv(csv_path)
    
    # Filtrar audios de la clase que buscamos aumentar
    class_df = df[df['class'] == class_to_augment]
    
    # Crear listas para almacenar las nuevas filas del CSV
    new_rows = []

    # Realizar augmentación para cada partición: train, validation, test
    for partition, n_augments in augment_counts.items():
        # Filtrar los audios de la partición correspondiente
        partition_df = class_df[class_df['partition'] == partition]
        
        # Seleccionar los audios al azar para aumentar
        audios_to_augment = partition_df.sample(n=n_augments)
        
        # Crear la carpeta si no existe
        new_folder = new_folders[partition]
        if not os.path.exists(new_folder):
            os.makedirs(new_folder)
        
        # Para cada audio seleccionado
        for i, row in audios_to_augment.iterrows():
            audio_path = row['path']  # Ruta del audio original
            augment_type = np.random.choice(['speed', 'pitch','distortion'])  # Elegir una técnica al azar XX modificar si se agregan mas aumetaciones
            
            # Generar un nuevo nombre para el archivo aumentado
            new_audio_name = f"{row['speaker_id']}_aug_{i}.wav"
            new_audio_path = os.path.join(new_folder, new_audio_name)
            
            # Aplicar augmentación
            augment_audio(audio_path, new_audio_path, augment_type)
            
            # Crear una nueva fila para el CSV
            new_row = row.copy()
            new_row['path'] = new_audio_path
            new_rows.append(new_row)  # Añadir la nueva fila a la lista
    
    # Concatenar las nuevas filas al DataFrame original
    new_df = pd.concat([df, pd.DataFrame(new_rows)], ignore_index=True)
    
    # Guardar el CSV actualizado
    new_df.to_csv(new_csv_path, index=False)
    print(f"Data augmentation completada. CSV actualizado en {new_csv_path}")

In [2]:
csv_path = "./labels.csv"  # Ruta del CSV original
new_csv_path = "./updated_csv.csv"  # Ruta para guardar el CSV actualizado

# Definir cuántos audios aumentar para cada set
augment_counts = {
    'train': 12,
    'validation': 6,
    'test': 8
}

# Rutas para las nuevas carpetas donde se guardarán los audios aumentados
new_folders = {
    'train': 'new_train_folder',
    'validation': 'new_validation_folder',
    'test': 'new_test_folder'
}

augment_class(csv_path, class_to_augment=0, augment_counts=augment_counts, new_folders=new_folders, new_csv_path=new_csv_path)

Data augmentation completada. CSV actualizado en ./updated_csv.csv
